<img src="images/nvidia_header.png" style="margin-left: -30px; width: 300px; float: left;">

# Week 1: Find Clusters of Infected People

<span style="color:red">
**URGENT WARNING**

We have been receiving reports from health facilities that a new, fast-spreading virus has been discovered in the population. To prepare our response, we need to understand the geospatial distribution of those who have been infected. Find out whether there are identifiable clusters of infected individuals and where they are.    
</span>

Your goal for this notebook will be to estimate the location of dense geographic clusters of infected people using incoming data from week 1 of the simulated epidemic.

## Imports

In [1]:
%load_ext cudf.pandas
import pandas as pd
import cuml

import cupy as cp

## Load Data

Begin by loading the data you've received about week 1 of the outbreak into a cuDF-accelerated pandas DataFrame. The data is located at `'./data/week1.csv'`. For this notebook you will only need the `'lat'`, `'long'`, and `'infected'` columns. Either drop the columns after loading, or use the `pd.read_csv` named argument `usecols` to provide a list of only the columns you need.

In [ ]:
df = pd.read_csv('./data/week1.csv')
df

,age,sex,lat,long,employment,infected
0,0,m,54.522510,-1.571896,U,False
1,0,m,54.554030,-1.524968,U,False
2,0,m,54.552486,-1.435203,U,False
3,0,m,54.537189,-1.566215,U,False
4,0,m,54.528212,-1.588462,U,False
...,...,...,...,...,...,...
58479889,90,f,51.634416,-2.925863,V,False
58479890,90,f,51.556972,-3.036290,V,False
58479891,90,f,51.588992,-2.921915,V,False
58479892,90,f,51.590974,-2.954539,V,False


## Make Data Frame of the Infected

Make a new DataFrame `infected_df` that contains only the infected members of the population.

**Tip**: Reset the index of `infected_df` with `.reset_index(drop=True)`. 

In [13]:
infected_df = df[df['infected'] == 1]
infected_df = infected_df.reset_index(drop=True)
infected_df

,age,sex,lat,long,employment,infected
0,44,f,54.472766,-1.654932,Q,True
1,47,f,54.529717,-1.667143,Q,True
2,47,f,54.512986,-1.589866,I,True
3,49,f,54.522322,-1.380694,Q,True
4,51,f,54.541660,-1.613490,Q,True
...,...,...,...,...,...,...
18143,56,f,52.428347,-3.322932,I,True
18144,59,f,52.415895,-3.263942,Q,True
18145,63,f,52.539934,-3.617128,Q,True
18146,63,f,52.435490,-3.597263,I,True


## Make Grid Coordinates for Infected Locations

Provided for you in the next cell (which you can expand by clicking on the "..." and contract again after executing by clicking on the blue left border of the cell) is the lat/long to OSGB36 grid coordinates converter you used earlier in the workshop. Use this converter to create grid coordinate values stored in `northing` and `easting` columns of the `infected_df` you created in the last step.

In [16]:
# https://www.ordnancesurvey.co.uk/docs/support/guide-coordinate-systems-great-britain.pdf

def latlong2osgbgrid_cupy(lat, long, input_degrees=True):
    '''
    Converts latitude and longitude (ellipsoidal) coordinates into northing and easting (grid) coordinates, using a Transverse Mercator projection.
    
    Inputs:
    lat: latitude coordinate (N)
    long: longitude coordinate (E)
    input_degrees: if True (default), interprets the coordinates as degrees; otherwise, interprets coordinates as radians
    
    Output:
    (northing, easting)
    '''
    
    if input_degrees:
        lat = lat * cp.pi/180
        long = long * cp.pi/180

    a = 6377563.396
    b = 6356256.909
    e2 = (a**2 - b**2) / a**2

    N0 = -100000 # northing of true origin
    E0 = 400000 # easting of true origin
    F0 = .9996012717 # scale factor on central meridian
    phi0 = 49 * cp.pi / 180 # latitude of true origin
    lambda0 = -2 * cp.pi / 180 # longitude of true origin and central meridian
    
    sinlat = cp.sin(lat)
    coslat = cp.cos(lat)
    tanlat = cp.tan(lat)
    
    latdiff = lat-phi0
    longdiff = long-lambda0

    n = (a-b) / (a+b)
    nu = a * F0 * (1 - e2 * sinlat ** 2) ** -.5
    rho = a * F0 * (1 - e2) * (1 - e2 * sinlat ** 2) ** -1.5
    eta2 = nu / rho - 1
    M = b * F0 * ((1 + n + 5/4 * (n**2 + n**3)) * latdiff - 
                  (3*(n+n**2) + 21/8 * n**3) * cp.sin(latdiff) * cp.cos(lat+phi0) +
                  15/8 * (n**2 + n**3) * cp.sin(2*(latdiff)) * cp.cos(2*(lat+phi0)) - 
                  35/24 * n**3 * cp.sin(3*(latdiff)) * cp.cos(3*(lat+phi0)))
    I = M + N0
    II = nu/2 * sinlat * coslat
    III = nu/24 * sinlat * coslat ** 3 * (5 - tanlat ** 2 + 9 * eta2)
    IIIA = nu/720 * sinlat * coslat ** 5 * (61-58 * tanlat**2 + tanlat**4)
    IV = nu * coslat
    V = nu / 6 * coslat**3 * (nu/rho - cp.tan(lat)**2)
    VI = nu / 120 * coslat ** 5 * (5 - 18 * tanlat**2 + tanlat**4 + 14 * eta2 - 58 * tanlat**2 * eta2)

    northing = I + II * longdiff**2 + III * longdiff**4 + IIIA * longdiff**6
    easting = E0 + IV * longdiff + V * longdiff**3 + VI * longdiff**5

    return(northing, easting)

In [ ]:
cupy_lat = cp.asarray(infected_df['lat'])
cupy_long = cp.asarray(infected_df['long'])

infected_df['northing'], infected_df['easting'] = latlong2osgbgrid_cupy(cupy_lat, cupy_long, input_degrees=1)
infected_df

,age,sex,lat,long,employment,infected,northing,easting
0,44,f,54.472766,-1.654932,Q,True,2.077547e+06,3.779862e+06
1,47,f,54.529717,-1.667143,Q,True,2.085387e+06,3.776817e+06
2,47,f,54.512986,-1.589866,I,True,2.083085e+06,3.777713e+06
3,49,f,54.522322,-1.380694,Q,True,2.084369e+06,3.777213e+06
4,51,f,54.541660,-1.613490,Q,True,2.087029e+06,3.776177e+06
...,...,...,...,...,...,...,...,...
18143,56,f,52.428347,-3.322932,I,True,1.789200e+06,3.879192e+06
18144,59,f,52.415895,-3.263942,Q,True,1.787404e+06,3.879736e+06
18145,63,f,52.539934,-3.617128,Q,True,1.805272e+06,3.874283e+06
18146,63,f,52.435490,-3.597263,I,True,1.790230e+06,3.878879e+06


## Find Clusters of Infected People

Use DBSCAN to find clusters of at least 25 infected people where no member is more than 2000m from at least one other cluster member. Create a new column in `infected_df` which contains the cluster to which each infected person belongs.

In [ ]:
#dbscan = cuml.<<<<FIXME>>>>
#infected_df['cluster'] = dbscan.<<<<FIXME>>>>(infected_df[<<<<FIXME>>>>])
#infected_df

dbscan = cuml.DBSCAN(min_samples=25, eps=2000)
infected_df['cluster'] = dbscan.fit_predict(infected_df[['northing', 'easting']])
infected_df

,age,sex,lat,long,employment,infected,northing,easting,cluster
0,44,f,54.472766,-1.654932,Q,True,2.077547e+06,3.779862e+06,0
1,47,f,54.529717,-1.667143,Q,True,2.085387e+06,3.776817e+06,0
2,47,f,54.512986,-1.589866,I,True,2.083085e+06,3.777713e+06,0
3,49,f,54.522322,-1.380694,Q,True,2.084369e+06,3.777213e+06,0
4,51,f,54.541660,-1.613490,Q,True,2.087029e+06,3.776177e+06,0
...,...,...,...,...,...,...,...,...,...
18143,56,f,52.428347,-3.322932,I,True,1.789200e+06,3.879192e+06,1
18144,59,f,52.415895,-3.263942,Q,True,1.787404e+06,3.879736e+06,1
18145,63,f,52.539934,-3.617128,Q,True,1.805272e+06,3.874283e+06,1
18146,63,f,52.435490,-3.597263,I,True,1.790230e+06,3.878879e+06,1


## Find the Centroid of Each Cluster

Use grouping to find the mean `northing` and `easting` values for each cluster identified above.

In [ ]:
#centroids_df = infected_df[['northing', 'easting', 'cluster']].groupby(<<<<FIXME>>>>).<<<<FIXME>>>>()
#centroids_df
centroids_df = infected_df[['northing', 'easting', 'cluster']].groupby('cluster')[['northing', 'easting']].mean()
centroids_df

,northing,easting
cluster,,
-1,1.956430e+06,3.820269e+06
0,2.079523e+06,3.779090e+06
1,1.923267e+06,3.835686e+06
2,2.058501e+06,3.787181e+06
3,1.747638e+06,3.891555e+06


Find the number of people in each cluster by counting the number of appearances of each cluster's label in the column produced by DBSCAN.

In [ ]:
infected_df['cluster'].value_counts()
#infected_df.groupby('cluster').size().sort_values()
#infected_df['cluster'].value_counts().max()
#infected_df['cluster'].value_counts().index.tolist()
#infected_df['cluster'].value_counts().idxmax()
#centroids_df.iloc[infected_df['cluster'].value_counts().idxmax()]
#centroids_df.loc[infected_df['cluster'].value_counts().idxmax()]
#round(centroids_df.loc[infected_df['cluster'].value_counts().idxmax()],2)
#infected_df[['cluster', 'northing', 'easting']].groupby(['cluster']).size

cluster
 1    18070
 0       47
-1       16
 3        8
 2        7
Name: count, dtype: int64

## Find the Centroid of the Cluster with the Most Members ##

Use the cluster label for with the most people to filter `centroid_df` and write the answer to `my_assessment/question_1.json`. 

In [ ]:
centroids_df.loc[infected_df['cluster'].value_counts().idxmax()].to_json('my_assessment/question_1.json')

/opt/conda/lib/python3.10/site-packages/cudf/io/json.py:194: UserWarning: Using CPU via Pandas to write JSON dataset
  warnings.warn("Using CPU via Pandas to write JSON dataset")


## Check Submission ##

In [80]:
!cat my_assessment/question_1.json

{"northing":1923267.3040023146,"easting":3835686.2515772688}

**Tip**: Your submission file should contain one line of text, similar to: 

```
{'northing':XXX.XX,'easting':XXX.XX}
```

<div align="center"><h2>Please Restart the Kernel</h2></div>

...before moving to the next notebook.

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

<img src="images/nvidia_header.png" style="margin-left: -30px; width: 300px; float: left;">